In [1]:
import pandas as pd

In [2]:
demographics = pd.read_csv('../data/demographics.csv', na_values = ['-2222', '-2222.2', '-2', '-9999'])
causes = pd.read_csv('../data/leading_causes_of_death.csv', na_values = ['-2222', '-2222.2', '-2', '-9999'])
deaths = pd.read_csv('../data/measureDeath_1.csv', na_values = ['-2222', '-2222.2', '-2', '-9999'])

In [3]:
causes.replace(to_replace = [-1111, -1111.1, -1], value = 0, inplace = True)
demographics.replace(to_replace = [-1111, -1111.1, -1], value = 0, inplace = True)
deaths.replace(to_replace = [-1111, -1111.1, -1], value = 0, inplace = True)
deaths = deaths.drop(columns = ['Unnamed: 0']).rename({
    'Brst_Cancer': 'breast_cancer',
    'Col_Cancer': 'colon_cancer',
    'Lung_Cancer': 'lung_cancer'}, axis = 1)
deaths.head()

,CHSI_County_Name,CHSI_State_Name,breast_cancer,colon_cancer,lung_cancer
0,Autauga,Alabama,34.1,15.9,65.3
1,Baldwin,Alabama,22.5,15.3,62.1
2,Barbour,Alabama,30.5,23.4,70.6
3,Bibb,Alabama,30.1,17.4,64.6
4,Blount,Alabama,30.2,16.8,71.7


In [4]:
# select necessary columns
causes_regex = ('CHSI_County_Name|'
                'CHSI_State_*?|'
                'breast_cancer|'
                'colon_cancer|'
                'lung_cancer|'
                '^[a-zA-Z]+_[a-zA-Z]+_Comp|'
                '^[a-zA-Z]+_[a-zA-Z]+_BirthDef|'
                '^[a-zA-Z]+_[a-zA-Z]+_Injury|'
                '^[a-zA-Z]+_[a-zA-Z]+_Homicide|'
                '^[a-zA-Z]+_[a-zA-Z]+_Suicide|'
                '^[a-zA-Z]+_[a-zA-Z]+_HeartDis|'
                '^[a-zA-Z]+_[a-zA-Z]+_HIV|')
causes = causes.filter(regex = causes_regex)

In [5]:
demographics = demographics[['CHSI_County_Name', 'CHSI_State_Name', 'CHSI_State_Abbr', 'Population_Size']]

In [6]:
merged = causes.merge(
    demographics, 
    on = ['CHSI_County_Name', 'CHSI_State_Name'], 
    how = 'inner').merge(
    deaths,
    on = ['CHSI_County_Name', 'CHSI_State_Name'],
    how = 'inner')
assert (merged['CHSI_State_Abbr_x'] == merged['CHSI_State_Abbr_y']).all()
merged['CHSI_State_Abbr'] = merged['CHSI_State_Abbr_x']
merged = merged.drop(columns = ['CHSI_State_Abbr_x', 'CHSI_State_Abbr_y'])

In [7]:
# sum over rows of each cause of death
merged['complication_of_pregnancy_birth'] = merged.filter(regex = 'Comp').sum(axis = 1)
merged['birth_defects'] = merged.filter(regex = 'BirthDef').sum(axis = 1)
merged['injury'] = merged.filter(regex = 'Injury').sum(axis = 1)
merged['homicide'] = merged.filter(regex = 'Homicide').sum(axis = 1)
merged['suicide'] = merged.filter(regex = 'Suicide').sum(axis = 1)
merged['heart_disease'] = merged.filter(regex = 'HeartDis').sum(axis = 1)
merged['hiv'] = merged.filter(regex = 'HIV').sum(axis = 1)
merged['breast_cancer'] = merged['breast_cancer'] * merged['Population_Size'] / 1e5
merged['colon_cancer'] = merged['colon_cancer'] * merged['Population_Size'] / 1e5
merged['lung_cancer'] = merged['lung_cancer'] * merged['Population_Size'] / 1e5
sum_cols = ['CHSI_County_Name', 'CHSI_State_Name', 'CHSI_State_Abbr', 'complication_of_pregnancy_birth',
           'birth_defects', 'injury', 'homicide', 'suicide', 'heart_disease', 'hiv', 
            'breast_cancer', 'colon_cancer', 'lung_cancer', 'Population_Size']
sum_data = merged[sum_cols]
sum_data = sum_data.rename({'Population_Size': 'population', 'CHSI_State_Abbr': 'state'}, axis = 1)
sum_data.head()

,CHSI_County_Name,CHSI_State_Name,state,complication_of_pregnancy_birth,birth_defects,injury,homicide,suicide,heart_disease,hiv,breast_cancer,colon_cancer,lung_cancer,population
0,Autauga,Alabama,AL,0.0,0.0,142.0,0.0,41.0,447.0,0.0,16.576692,7.729308,31.743636,48612
1,Baldwin,Alabama,AL,171.0,64.0,318.0,63.0,38.0,455.0,0.0,36.581850,24.875658,100.965906,162586
2,Barbour,Alabama,AL,0.0,0.0,113.0,0.0,55.0,464.0,0.0,8.666270,6.648876,20.060284,28414
3,Bibb,Alabama,AL,0.0,0.0,347.0,0.0,76.0,421.0,0.0,6.476316,3.743784,13.899336,21516
4,Blount,Alabama,AL,103.0,103.0,258.0,0.0,40.0,332.0,0.0,16.828950,9.361800,39.954825,55725


In [8]:
# each column besides CHSI_State_Abbr is the death rate of that cause per 100000 population
agg_data = sum_data.groupby('state', as_index = False).sum()[[
    'state', 'complication_of_pregnancy_birth', 'birth_defects', 'injury', 'homicide', 'suicide',
    'heart_disease', 'hiv', 'breast_cancer', 'colon_cancer', 'lung_cancer', 'population'
]]
for col_name in agg_data.columns:
    if col_name != 'population' and col_name != 'state':
        agg_data[col_name] = agg_data[col_name] / agg_data['population'] * 1e5
agg_data = agg_data.drop(columns = ['population'])
agg_data.head()

,state,complication_of_pregnancy_birth,birth_defects,injury,homicide,suicide,heart_disease,hiv,breast_cancer,colon_cancer,lung_cancer
0,AK,68.860457,80.462766,724.164897,21.245787,339.631227,889.008093,0.000000,19.335803,18.992446,56.258729
1,AL,133.112233,50.901662,416.845115,54.258538,44.736417,671.353423,6.428529,26.605960,19.052720,63.059479
2,AR,99.382762,45.301556,641.202323,49.043702,121.331887,956.334194,9.535276,24.595171,21.571900,68.544220
3,AZ,43.877284,18.217660,140.218733,16.247728,27.511697,113.094288,0.168370,23.273922,17.357077,46.944131
4,CA,35.010375,16.677669,77.197184,16.121378,8.211524,81.755452,0.907779,24.088078,17.279528,45.326902


In [9]:
agg_data.to_csv('../data/rates_causes_of_death_bystate.csv', index = False)